<a href="https://colab.research.google.com/github/jvishnuvardhan/Advanced-Deep-Learning-with-Keras/blob/master/custom_training_model_cloning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!pip install tensorflow==2.2-rc2

In [8]:
import tensorflow as tf 
tf.__version__

'2.2.0-rc2'

In [9]:
import tensorflow as tf
#tf.enable_eager_execution()
tf.keras.backend.set_floatx("float64")
import numpy as np
import tensorflow.keras.backend as K
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.losses import binary_crossentropy


# Input layer, one hidden layer
input_layer = Input(batch_shape=(None, 20))
dense_1 = Dense(1028)(input_layer)
output_2 = Dense(1, activation="sigmoid")(dense_1)
model = Model(inputs=input_layer, outputs= output_2)
print(model.summary())
def custom_loss2(output2, label2):
    loss_value = K.mean(binary_crossentropy(label2, output2)) 
    return loss_value
def EWC_loss(new_weights, old_weights, fisher_matrix, rate):
    sum_w = 0
    for v in range(len(fisher_matrix)):
        sum_w += tf.reduce_sum(tf.multiply(fisher_matrix[v], tf.square(new_weights[v] - old_weights[v]))) 
    return sum_w*rate

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 20)]              0         
_________________________________________________________________
dense_2 (Dense)              (None, 1028)              21588     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 1029      
Total params: 22,617
Trainable params: 22,617
Non-trainable params: 0
_________________________________________________________________
None


In [0]:
n_sample = 1000
fix = np.array([range(n_sample),]*20).transpose()
features = np.cos(fix + np.random.rand(n_sample,20))
labels_2 = np.cos(np.array([range(n_sample),]*1).transpose() + np.random.rand(n_sample,1))
labels_2 = np.array([labels_2>=0]).astype(float).reshape(len(labels_2),1)

In [11]:
#del optimizer
#del tape
num_epochs = 1000
optimizer = tf.keras.optimizers.SGD()
ewc_model = tf.keras.models.clone_model(model)
ewc_model.set_weights(model.get_weights()) # add this line to use exactly same weights
old_weights = model.trainable_variables.copy()
fisher_matrix = old_weights
for epoch in range(num_epochs):
    with tf.GradientTape() as tape:
        out = ewc_model(features)
        new_weights = ewc_model.trainable_variables.copy()
        ewc_loss = EWC_loss(new_weights, old_weights, fisher_matrix, 0.5)
        loss = ewc_loss + custom_loss2(labels_2, out)
        grad = tape.gradient(loss, ewc_model.trainable_variables)
        optimizer.apply_gradients(grads_and_vars=zip(grad, ewc_model.trainable_variables))
    if (epoch+1)%100 == 0:
        print("epch: {}, loss: {}".format(epoch, loss.numpy()))
        print(ewc_loss.numpy(), loss.numpy())

epch: 99, loss: 1.2475948295218897
-0.002393169600546847 1.2475948295218897
epch: 199, loss: 1.1811368401223423
-0.0068271250011793065 1.1811368401223423
epch: 299, loss: 1.145449661763205
-0.012639246403220715 1.145449661763205
epch: 399, loss: 1.118805455672477
-0.019696919448428583 1.118805455672477
epch: 499, loss: 1.0958003083925993
-0.02797807437857744 1.0958003083925993
epch: 599, loss: 1.074304706440945
-0.03751690775620519 1.074304706440945
epch: 699, loss: 1.0532353922466104
-0.04838604792113444 1.0532353922466104
epch: 799, loss: 1.0319290881777619
-0.060689650699999015 1.0319290881777619
epch: 899, loss: 1.0099128086305043
-0.07456105269238739 1.0099128086305043
epch: 999, loss: 0.9868029803402413
-0.09016282149003818 0.9868029803402413


In [12]:
del optimizer
del tape
num_epochs = 1000
optimizer = tf.keras.optimizers.SGD()
ewc_model = tf.keras.models.clone_model(model) 
ewc_model.set_weights(model.get_weights()) # add this line to use exactly same weights
old_weights = model.trainable_variables.copy()
for epoch in range(num_epochs):
    with tf.GradientTape() as tape:
        out = ewc_model(features)
        new_weights = ewc_model.trainable_variables.copy()
        ewc_loss = 0.5*EWC_loss(new_weights, old_weights, fisher_matrix, 1.0)
        loss = ewc_loss + custom_loss2(labels_2, out)
        grad = tape.gradient(loss, ewc_model.trainable_variables)
        optimizer.apply_gradients(grads_and_vars=zip(grad, ewc_model.trainable_variables))
    if (epoch+1)%100 == 0:
        print("epch: {}, loss: {}".format(epoch, loss.numpy()))
        print(ewc_loss.numpy(), loss.numpy())

epch: 99, loss: 1.2475948295218897
-0.002393169600546847 1.2475948295218897
epch: 199, loss: 1.1811368401223423
-0.0068271250011793065 1.1811368401223423
epch: 299, loss: 1.145449661763205
-0.012639246403220715 1.145449661763205
epch: 399, loss: 1.118805455672477
-0.019696919448428583 1.118805455672477
epch: 499, loss: 1.0958003083925993
-0.02797807437857744 1.0958003083925993
epch: 599, loss: 1.074304706440945
-0.03751690775620519 1.074304706440945
epch: 699, loss: 1.0532353922466104
-0.04838604792113444 1.0532353922466104
epch: 799, loss: 1.0319290881777619
-0.060689650699999015 1.0319290881777619
epch: 899, loss: 1.0099128086305043
-0.07456105269238739 1.0099128086305043
epch: 999, loss: 0.9868029803402413
-0.09016282149003818 0.9868029803402413
